# PyBaMM

[PyBaMM](https://docs.pybamm.org) (Python Battery Mathematical Modelling) is an open-source framework for simulating battery cells, providing a variety of different mathematical models and parameterizations based on real-world cells.

In this example, we show an examplary way to integrate PyBaMM models into a Vessim Simulation by implementing our `vs.Storage` Interface.

In [1]:
from __future__ import annotations
from typing import Optional

import pybamm    # version >= 24.5
import numpy as np
import vessim as vs

# Hotfix to execute asyncio in Jupyter
import nest_asyncio
nest_asyncio.apply()

2024-07-29 16:42:43.884 - [WARNING] citations.read_citations(76): Citations could not be read because the 'pybtex' library is not installed. Install 'pybamm[cite]' to enable citation reading.


In [2]:
class PybammBattery(vs.Storage):
    """Experimental class to integrate PyBaMM simulations for batteries into Vessim.

    Args:
        model_type: Type of model to be used for simulation. This class only supports lithium-ion
            models like SPM, SPMe, DFN.
        parameter_values: pybamm.ParameterValues that parameterize the model.
        number_of_cells: The number of cells in the battery pack. Even though only a single cell is
            simulated, this parameter scales the cell to a battery pack. Defaults to 1.
        initial_soc: Initial battery state-of-charge. Has to be between 0 and 1. Defaults to 0.
            Note that there might be a slight difference between the initial state-of-charge and
            the estimated state-of-charge after initialization.
        min_soc: Minimum allowed battery state-of-charge. Has to be between 0 and 1. If the
            battery's state-of-charge is below minimum, only charging is allowed. Note, that the
            condition is checked before the battery simulation is stepped, which might result in
            the battery's SoC falling under said value during a time-step. Can be changed during
            simulation. Defaults to 0.
        max_charge_rate: Limits the charging current of the battery. Should be set based on
            modeled cell / the used parameterization. Defaults to 0.7C for the LGM50 21700 cell /
            Chen2020 parameter set.
        max_discharge_rate: Limits the discharging current of the battery. Should be set based on
            modeled cell / the used parameterization. Defaults to -1.5C for the LGM50 21700 cell /
            Chen2020 parameter set.
        output_variables: Variables of the PyBaMM solution to be returned by the state function.
        min_charge_current: Current, at which point battery stops charging when almost full.
            Defaults to 0.05A.
        min_discharge_current: Current, at which point battery stops discharging when almost empty.
            Defaults to -0.05A.
        options: Options for the PyBaMM model. Class automatically sets the `operating mode` to
            `power` to enable power-based control. Defaults to None.
        geometry: Optional geometry upon which to solve the model as passed to the PyBaMM
            simulation. Defaults to None.
        submesh_types: Optional dictionary of the types of submesh to use on each subdomain as
            passed to the PyBaMM simulation. Defaults to None.
        var_pts: Optional dictionary of the number of points used by each spatial variable as
            passed to the PyBaMM simulation. Defaults to None.
        spatial_methods: Optional dictionary of the types of spatial method to use on each
            domain as passed to the PyBaMM simulation. Defaults to None.
        solver: Optional Solver to use to solve the model as passed to the PyBaMM simulation.
            Defaults to None.
    """

    def __init__(
        self,
        model_type: type[pybamm.lithium_ion.BaseModel],
        parameter_values: pybamm.ParameterValues,
        number_of_cells: int = 1,
        initial_soc: float = 0,
        min_soc: float = 0,
        max_charge_rate: float = 0.7,
        max_discharge_rate: float = -1.5,
        output_variables: Optional[list] = None,
        min_charge_current: float = 0.05,
        min_discharge_current: float = -0.05,
        options: Optional[dict] = None,
        geometry: Optional[pybamm.Geometry] = None,
        submesh_types: Optional[dict] = None,
        var_pts: Optional[dict] = None,
        spatial_methods: Optional[dict] = None,
        solver: Optional[pybamm.BaseSolver] = None,
    ) -> None:
        assert number_of_cells > 0, "There has to be a positive number of cells."
        self.number_of_cells = number_of_cells
        assert 0 <= initial_soc <= 1
        assert 0 <= min_soc <= 1
        self.min_soc = min_soc

        # Run experiment to retrieve charging power limits based on the Open-Circuit Voltage
        # These power limitations take voltage and current limits into account
        v_cut_lower = parameter_values["Lower voltage cut-off [V]"]
        v_cut_higher = parameter_values["Upper voltage cut-off [V]"]
        experiment = pybamm.Experiment(
            [
                (
                    f"Charge at {max_charge_rate}C until {v_cut_higher}V",
                    f"Hold at {v_cut_higher}V until {min_charge_current}A",
                )
            ],
            period="1 second",
        )
        charging_sim = pybamm.Simulation(
            experiment=experiment,
            model=model_type(),
            geometry=geometry,
            parameter_values=parameter_values.copy(),
            submesh_types=submesh_types,
            var_pts=var_pts,
            spatial_methods=spatial_methods,
            solver=solver,
        )
        sol = charging_sim.solve(initial_soc=0)
        self._ocv_charging = sol["Battery open-circuit voltage [V]"].data
        self._max_power_charging = -sol["Power [W]"].data

        # Run experiment to retrieve discharging power limits based on the Open-Circuit Voltage
        # These power limitations take voltage and current limits into account
        experiment = pybamm.Experiment(
            [
                (
                    f"Discharge at {-max_discharge_rate}C until {v_cut_lower}V",
                    f"Hold at {v_cut_lower}V until {-min_discharge_current}A",
                )
            ],
            period="1 second",
        )
        discharging_sim = pybamm.Simulation(
            experiment=experiment,
            model=model_type(options=options),
            geometry=geometry,
            parameter_values=parameter_values.copy(),
            submesh_types=submesh_types,
            var_pts=var_pts,
            spatial_methods=spatial_methods,
            solver=solver,
        )
        sol = discharging_sim.solve(initial_soc=1)
        self._ocv_discharging = sol["Battery open-circuit voltage [V]"].data[::-1]
        self._max_power_discharging = sol["Power [W]"].data[::-1]

        # Run experiment to retrieve State-of-Charge estimations based on the Open-Circuit Voltage
        # Because the experiment takes longer than 24h, at least Pybamm version 24.5 has to be used
        experiment = pybamm.Experiment(
            [
                (
                    f"Discharge at {-min_discharge_current * v_cut_lower}W for 500 hours or until {v_cut_lower}V",
                )
            ],
            period="1 minute",
        )
        soc_sim = pybamm.Simulation(
            model=model_type(options=options),
            experiment=experiment,
            geometry=geometry,
            parameter_values=parameter_values.copy(),
            submesh_types=submesh_types,
            var_pts=var_pts,
            spatial_methods=spatial_methods,
            solver=solver,
        )
        sol = soc_sim.solve(initial_soc=1)
        self._ocv_values = sol["Battery open-circuit voltage [V]"].data[::-1]
        self._soc_values = np.linspace(0.0, 1.0, self._ocv_values.size)

        # Use power-based control instead of current-based control
        if not options:
            options = {"operating mode": "power"}
        else:
            options["operating mode"] = "power"
        parameter_values.update({"Power function [W]": "[input]"}, check_already_exists=False)

        # Build simulation of single cell and get initial solution
        # Pybamm throws errors when doing the small initial step if the initial soc is 1 or 0
        # due to the fact that setting the power function to 0.0 does not guarantee that no current
        # is applied, sometimes resulting in the exceeding of voltage limits
        self.sim = pybamm.Simulation(
            model=model_type(options),
            geometry=geometry,
            parameter_values=parameter_values,
            submesh_types=submesh_types,
            var_pts=var_pts,
            spatial_methods=spatial_methods,
            solver=solver,
        )
        if initial_soc > 0.999:
            initial_soc = 0.999
        if initial_soc < 0.001:
            initial_soc = 0.001
        self.sim.build(initial_soc=initial_soc)
        self.cell_solution = self.sim.step(dt=1e-6, inputs={"Power function [W]": 0.0}).last_state

        # Model variables to be returned by the state function
        if output_variables is None:
            self.output_variables = []
        else:
            self.output_variables = output_variables

    def soc(self) -> float:
        """Determines the State-of-Charge of the battery based on the Open-Circuit Voltage."""
        value = self.cell_solution["Battery open-circuit voltage [V]"].data[0]
        idx = np.searchsorted(self._ocv_values, value, side="right")
        if idx == 0:
            # Battery is empty (normally not reached due to voltage limits)
            return 0.0
        elif idx == len(self._ocv_values):
            # Battery is full (normally not reached due to voltage limits)
            return 1.0
        else:
            # SoC estimation based on the earlier experiment
            x1, x2 = self._ocv_values[idx - 1], self._ocv_values[idx]
            y1, y2 = self._soc_values[idx - 1], self._soc_values[idx]
            return y1 + (value - x1) * (y2 - y1) / (x2 - x1)

    def update(self, power: float, duration: int) -> float:
        """Steps the pybamm simulation for a specific duration based on the applied power."""
        value = self.cell_solution["Battery open-circuit voltage [V]"].data[0]

        # Don't exceed minimum state-of-charge
        if self.soc() < self.min_soc and power < 0.0:
            power = 0.0

        # Determine if power limits are applicable based on the Open-Circuit Voltage
        if power > 0.0:
            # Limit charging power based on charging experiment
            idx = np.searchsorted(self._ocv_charging, value, side="right") + duration
            if idx >= len(self._ocv_charging):
                power = 0.0
            elif power > self._max_power_charging[idx] * self.number_of_cells:
                power = self._max_power_charging[idx] * self.number_of_cells
        elif power < 0.0:
            # Limit discharging power based on discharging experiment
            idx = np.searchsorted(self._ocv_discharging, value, side="left") - duration
            if idx <= 0:
                power = 0.0
            elif power < -self._max_power_discharging[idx] * self.number_of_cells:
                power = -self._max_power_discharging[idx] * self.number_of_cells

        # Step the electrochemical system of the modelled battery cell (scaled by number of cells)
        self.cell_solution = self.sim.step(
            duration, inputs={"Power function [W]": -power / self.number_of_cells}
        ).last_state

        # Return (dis-)charged energy
        return power * duration

    def state(self) -> dict:
        """Returns the state of the battery."""
        state = {"soc": self.soc(), "min_soc": self.min_soc}

        # Add variables to be watched to the state
        # For many parameters like currents and powers, the sign is flipped compared to vessim
        # E.g. - 5.0 W means charging of 5W in pybamm (when it would mean discharging in vessim)
        for variable in self.output_variables:
            state[f"Cell {variable}"] = self.cell_solution[variable].data[0]
        return state

We can add such a `PybammBattery` to a Vessim environment by simply passing the initialized object to the `add_microgrid` function. Simulations that use the `PybammBattery` should generally not use a large step-size.

In [3]:
environment = vs.Environment(sim_start="2022-06-09 00:00:00")

environment.add_microgrid(
    actors=[
        vs.ComputingSystem(
            nodes=[vs.MockSignal(value=200), vs.MockSignal(value=250)], pue=1.6
        ),
        vs.Actor(
            name="solar",
            signal=vs.HistoricalSignal.load(
                "solcast2022_global", column="Berlin", params={"scale": 5000}
            ),
        ),
    ],
    controllers=[vs.Monitor(outfile="result.csv")],
    storage=PybammBattery(
        model_type=pybamm.lithium_ion.SPMe,
        parameter_values=pybamm.ParameterValues("Chen2020"),
        initial_soc=0.8,
        min_soc=0.5,
        number_of_cells=80,
        output_variables=["Current [A]", "Voltage [V]", "Battery open-circuit voltage [V]"]
    ),
    step_size=60,
)

environment.run(until=24 * 3600)

2024-07-29 16:42:53.342 | INFO     | mosaik.scenario:start:311 - Starting "Actor" as "ComputingSystem-0" ...
2024-07-29 16:42:53.343 | INFO     | mosaik.scenario:start:311 - Starting "Actor" as "solar" ...
2024-07-29 16:42:53.343 | INFO     | mosaik.scenario:start:311 - Starting "Grid" as "Grid-0" ...
2024-07-29 16:42:53.344 | INFO     | mosaik.scenario:start:311 - Starting "Controller" as "Monitor-0" ...
2024-07-29 16:42:53.344 | INFO     | mosaik.scenario:start:311 - Starting "Storage" as "Storage-0" ...
2024-07-29 16:42:53.353 | INFO     | mosaik.scenario:run:651 - Starting simulation.
100%|██████████| 86400/86400 [00:13<00:00, 6412.15steps/s]
2024-07-29 16:43:06.830 | INFO     | mosaik.scenario:run:708 - Simulation finished successfully.


In [7]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv("result.csv", index_col=0)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index, y=df["storage.soc"]*100, fill="tozeroy", line_color="gray", name="SoC"
    ),
)
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["storage.min_soc"]*100,
        line_dash="dot",
        line_color="black",
        name="min. SoC",
    ),
)
fig.update_yaxes(title_text="State-of-Charge", range=[0, 100], ticksuffix="%")

fig.update_layout(
    width=600,
    height=300,
    margin={"r": 0, "l": 0, "t": 0.1, "b": 0},
    paper_bgcolor="white", plot_bgcolor="white",
    font_size=12,
    legend={"xanchor": "left", "x": 0.0},
)
fig.update_xaxes(showline=True, linewidth=1, linecolor="black", ticks="outside")
fig.update_yaxes(showline=True, linewidth=1, linecolor="black", ticks="outside", title_font_size=14)
fig.show()